# Gibbs Sampler

DS 5001 Text as Data

Lab 3/6

**Purpose:** We develop a simple topic modeler using collapsed Gibbs sampling as described by [Griffiths and Steyvers (2004)](https://collab.its.virginia.edu/access/content/group/b9e58ce7-0f44-48fe-9861-b7a7657f551a/Articles/sciencetopics.pdf).

# Setup

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
from nltk.corpus import stopwords 

# Convert F1 Corpus to F2

We want to convert any given F1 corpus (DOC) into unannotated TOKEN and VOCAB tables.

This is so we can work with ad hoc training data.

In [3]:
class Corpus:

    def __init__(self, doc_list:list, doc_col='doc_str'):
        "Create DOC table from F1 list"
        self.DOC = pd.DataFrame(doc_list, columns=[doc_col])
        self.DOC.index.name = 'doc_id'
        self.stop_words = set(stopwords.words('english')) 
        
    def convert_corpus(self):        
        "Convert raw docs into TOKEN and BOW tables"
        tokens = []
        for i, row in self.DOC.iterrows():
            for j, token in enumerate(row.doc_str.split()):
                term_str = re.sub(r'[\W_]+', '', token).lower()
                if term_str not in self.stop_words:
                    tokens.append((i, j, term_str))
        self.TOKEN = pd.DataFrame(tokens, columns=['doc_id','token_num','term_str'])\
            .set_index(['doc_id','token_num'])
        self.BOW = self.TOKEN.groupby(['doc_id','term_str']).term_str.count().to_frame('n')
        return self
        
    def extract_vocab(self):
        "Extract vocabulary VOCAB"
        self.VOCAB = self.TOKEN.term_str.value_counts().to_frame('n')
        self.VOCAB.index.name = 'term_str'   
        return self 

In [4]:
raw_docs = """
I ate a banana and a spinach smoothie for breakfast.
I like to eat broccoli and bananas.
Chinchillas and kittens are cute.
My sister adopted a kitten yesterday.
Look at this cute hamster munching on a piece of broccoli.
""".split("\n")[1:-1]

In [5]:
corpus1 = Corpus(raw_docs).convert_corpus().extract_vocab()

In [6]:
corpus1.BOW

n
doc_id term_str      
0      ate          1
       banana       1
       breakfast    1
       smoothie     1
       spinach      1
1      bananas      1
       broccoli     1
       eat          1
       like         1
2      chinchillas  1
       cute         1
       kittens      1
3      adopted      1
       kitten       1
       sister       1
       yesterday    1
4      broccoli     1
       cute         1
       hamster      1
       look         1
       munching     1
       piece        1

# Gibbs Sampler

We sample each document and word combination in the BOW table. In each case,
we are looking for two values:

* the topic with which a word has been most frequently labeled
* the topic with which the document has the most labeled words

We combine these values in order to align the label of the current word with the rest of the data.\
If a the topic is highly associated with both the word and the document, then that topic will get a high value.

Note that all that is going on here is a sorting operation -- the random assignment does not predict anything.\
Instead, we are just gathering words under topics and topics under documents.

**From Darling 2011:**
<hr />
<div style="float:left;">
<img src="images/gibbs-algo-text.png" width="650px" />
<img src="images/gibbs-algo.png" width="650px" />
</div>

In [42]:
class GibbsSampler:

    def __init__(self, n_topics=10, iters=100, a = .1, b = .1):

        # Map arguments
        self.n_topics = n_topics
        self.iters = iters
        self.a = a
        self.b = b 
        
        # Define topic table
        topic_names = [f"T{str(t).zfill(len(str(self.n_topics)))}" for t in range(self.n_topics)]
        self.TOPIC = pd.DataFrame({'top_terms':'TBD'}, index=topic_names)
        self.TOPIC.index.name = 'topic_id'

    def add_corpus(self, corpus:Corpus):
        "Add Corpus object"
        
        # Copy BOW and assign random topics        
        self.BOW = corpus.BOW.copy()
        self.BOW['topic_id'] = self.TOPIC.sample(len(self.BOW), replace=True).index
        
        # Get vocab length
        self.VOCAB = corpus.VOCAB
        self.W = self.VOCAB.shape[0]       
        
        return self
            
    def compute_topics(self):
        "Implement Collapsed Gibbs Sampling"
        
        # Create count tables
        self.THETA = self.BOW.groupby(['doc_id', 'topic_id']).size().unstack(fill_value=0)
        self.PHI = self.BOW.groupby(['topic_id', 'term_str']).size().unstack(fill_value=0)
        self.TOPIC['n'] = self.BOW['topic_id'].value_counts().reindex(self.TOPIC.index, fill_value=0)

        # Iterate
        for i in tqdm(range(self.iters)):

            # Estimate topic per word
            for (doc_id, term_str), row in self.BOW.iterrows():

                # Get the current topic
                z = row['topic_id']

                # Decrement counts
                self.THETA.at[doc_id, z] -= 1
                self.PHI.at[z, term_str] -= 1
                self.TOPIC.at[z, 'n'] -= 1
                self.W -= 1
    
                # How much does the topic like the word?
                doc_topic_prob = (self.THETA.loc[doc_id] + self.a) / (self.THETA.loc[doc_id].sum() + self.a * self.n_topics)                

                # How much does teh document like the topic?
                topic_term_prob = (self.PHI[term_str] + self.b) / (self.TOPIC['n'] + self.b * self.W) 

                # Compute topic probabilities
                PZ = doc_topic_prob * topic_term_prob
                PZ /= PZ.sum()  # Normalize

                # Sample new topic
                z_new = PZ.sample(weights=PZ).index[0]
                self.BOW.at[(doc_id, term_str), 'topic_id'] = z_new

                # Increment counts
                self.THETA.at[doc_id, z_new] += 1
                self.PHI.at[z_new, term_str] += 1
                self.TOPIC.at[z_new, 'n'] += 1
                self.W += 1
                
        return self
        
    def get_top_terms(self):
        "Get top words for each topic"
        for topic_id in self.TOPIC.index:
            self.TOPIC.loc[topic_id, 'top_terms'] = ' '.join(self.PHI.loc[topic_id, self.PHI.loc[topic_id] > 0].index.values)
            
        return self

In [43]:
def do_all(f1_list:[], k=4, iters=100):
    corpus = Corpus(f1_list).convert_corpus().extract_vocab()
    model = GibbsSampler(n_topics=k, iters=iters, a=1, b=1)\
        .add_corpus(corpus)\
        .compute_topics()\
        .get_top_terms()
    corpus = corpus.DOC.join(model.THETA.astype('int'))
    return corpus, model

# Demo 1

We use a toy example to see if the method works.\
Because our codd is not vert efficient, we just 

## Data

A small F1 corpus.

In [44]:
raw_docs = """
I ate a banana and a spinach smoothie for breakfast.
I like to eat broccoli and bananas.
Chinchillas and kittens are cute.
My sister adopted a kitten yesterday.
Look at this cute hamster munching on a piece of broccoli.
""".split("\n")[1:-1]

## Process

In [45]:
cp1, tm1 = do_all(raw_docs, k=5, iters=500)

100%|██████████| 500/500 [00:12<00:00, 40.56it/s]


In [46]:
tm1.BOW

n topic_id
doc_id term_str               
0      ate          1       T4
       banana       1       T0
       breakfast    1       T2
       smoothie     1       T4
       spinach      1       T3
1      bananas      1       T0
       broccoli     1       T2
       eat          1       T2
       like         1       T2
2      chinchillas  1       T1
       cute         1       T4
       kittens      1       T3
3      adopted      1       T4
       kitten       1       T4
       sister       1       T4
       yesterday    1       T2
4      broccoli     1       T0
       cute         1       T3
       hamster      1       T0
       look         1       T2
       munching     1       T1
       piece        1       T2

In [47]:
tm1.PHI

term_str,adopted,ate,banana,bananas,breakfast,broccoli,chinchillas,cute,eat,hamster,kitten,kittens,like,look,munching,piece,sister,smoothie,spinach,yesterday
topic_id,,,,,,,,,,,,,,,,,,,,
T0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
T1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
T2,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,1,0,0,0,1
T3,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0
T4,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0


In [48]:
tm1.TOPIC

,top_terms,n
topic_id,,
T0,banana bananas broccoli hamster,4
T1,chinchillas munching,2
T2,breakfast broccoli eat like look piece yesterday,7
T3,cute kittens spinach,3
T4,adopted ate cute kitten sister smoothie,6


In [49]:
tm1.THETA

topic_id,T0,T1,T2,T3,T4
doc_id,,,,,
0,1,0,1,1,2
1,1,0,3,0,0
2,0,1,0,1,1
3,0,0,1,0,3
4,2,1,2,1,0


In [50]:
cp1.style.background_gradient(axis=None)

,doc_str,T0,T1,T2,T3,T4
doc_id,,,,,,
0,I ate a banana and a spinach smoothie for breakfast.,1,0,1,1,2
1,I like to eat broccoli and bananas.,1,0,3,0,0
2,Chinchillas and kittens are cute.,0,1,0,1,1
3,My sister adopted a kitten yesterday.,0,0,1,0,3
4,Look at this cute hamster munching on a piece of broccoli.,2,1,2,1,0


# Demo 2

## Data

In [51]:
some_documents = [
    ["Hadoop", "Big Data", "HBase", "Java", "Spark", "Storm", "Cassandra"],
    ["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"],
    ["Python", "scikit-learn", "scipy", "numpy", "statsmodels", "pandas"],
    ["R", "Python", "statistics", "regression", "probability"],
    ["machine learning", "regression", "decision trees", "libsvm"],
    ["Python", "R", "Java", "C++", "Haskell", "programming languages"],
    ["statistics", "probability", "mathematics", "theory"],
    ["machine learning", "scikit-learn", "Mahout", "neural networks"],
    ["neural networks", "deep learning", "Big Data", "artificial intelligence"],
    ["Hadoop", "Java", "MapReduce", "Big Data"],
    ["statistics", "R", "statsmodels"],
    ["C++", "deep learning", "artificial intelligence", "probability"],
    ["pandas", "R", "Python"],
    ["databases", "HBase", "Postgres", "MySQL", "MongoDB"],
    ["libsvm", "regression", "support vector machines"]
]
raw_docs2  = [' '.join(item) for item in some_documents]

## Process

In [ ]:
cp2, tm2 = do_all(raw_docs2, k=10, iters=500)

 45%|████▌     | 226/500 [00:20<00:25, 10.82it/s]

In [ ]:
tm2.TOPIC

In [ ]:
cp2.style.background_gradient(axis=None)